In [1]:
# python libraries
import os
import json
import random
import math
import time
import datetime
import logging
from glob import glob

import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt

# d2 libraries
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg, LazyConfig, instantiate
from detectron2.utils import comm
from detectron2.utils.logger import log_every_n_seconds
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.engine.hooks import EvalHook, BestCheckpointer
from detectron2.data import (
    MetadataCatalog,
    DatasetCatalog,
    build_detection_train_loader,
    build_detection_test_loader,
    get_detection_dataset_dicts,
    DatasetMapper,
)
import detectron2.data.transforms as T
from detectron2.utils.logger import setup_logger

# my stuff
from data_utils import (
    read_split_file,
    register_dataset,
    random_split_mixed_set,
    random_split_ut_west_campus_set
)

# setup D2 logger
setup_logger()

%matplotlib inline

In [2]:
random_split_mixed_set(img_dir="data/panels/mixed", split_ratio=(0.7, 0.1, 0.2), seed=10)
mixed_datasets = read_split_file("data/panels/mixed/split.txt")

In [3]:
random_split_ut_west_campus_set("data/panels/ut_west_campus", split_ratio=(0.7, 0.1, 0.2), seed=10)
ut_west_campus_datasets = read_split_file("data/panels/ut_west_campus/split.txt")

In [ ]:
classes = ["label", "button"]
colors = [(0, 255, 0), (0, 0, 255)]

dataset_names = [
    "mixed_train",
    "mixed_val",
    "mixed_test",
    "ut_west_campus_train",
    "ut_west_campus_val",
    "ut_west_campus_test",
]

# register combined training and validation sets
for name, im_paths in zip(dataset_names, mixed_datasets + ut_west_campus_datasets):
    DatasetCatalog.register(
        name=name, func=lambda im_paths=im_paths: register_dataset(im_paths=im_paths)
    )
    MetadataCatalog.get(name=name).set(thing_classes=classes, thing_colors=colors)

elevator_metadata = MetadataCatalog.get("mixed_train")

In [ ]:
mixed_train = DatasetCatalog.get("mixed_train")
ut_west_campus_train = DatasetCatalog.get("ut_west_campus_train")
trainset = mixed_train + ut_west_campus_train
print("total trainset len: ", len(trainset))

In [ ]:
for d in random.sample(trainset, 5):
    img = cv2.imread(d["file_name"])
    print(d["file_name"])
    visualizer = Visualizer(
        img[:, :, ::-1],
        metadata=elevator_metadata,
        scale=1,
        instance_mode=ColorMode.SEGMENTATION,
    )
    out = visualizer.draw_dataset_dict(d)
    plt.imshow(out.get_image())
    plt.axis("off")
    plt.show()

In [ ]:
def do_train(args, cfg):
    """
    Args:
        cfg: an object with the following attributes:
            model: instantiate to a module
            dataloader.{train,test}: instantiate to dataloaders
            dataloader.evaluator: instantiate to evaluator for test set
            optimizer: instantaite to an optimizer
            lr_multiplier: instantiate to a fvcore scheduler
            train: other misc config defined in `configs/common/train.py`, including:
                output_dir (str)
                init_checkpoint (str)
                amp.enabled (bool)
                max_iter (int)
                eval_period, log_period (int)
                device (str)
                checkpointer (dict)
                ddp (dict)
    """
    model = instantiate(cfg.model)
    logger = logging.getLogger("detectron2")
    logger.info("Model:\n{}".format(model))
    model.to(cfg.train.device)

    cfg.optimizer.params.model = model
    optim = instantiate(cfg.optimizer)

    train_loader = instantiate(cfg.dataloader.train)

    model = create_ddp_model(model, **cfg.train.ddp)
    trainer = (AMPTrainer if cfg.train.amp.enabled else SimpleTrainer)(model, train_loader, optim)
    checkpointer = DetectionCheckpointer(
        model,
        cfg.train.output_dir,
        trainer=trainer,
    )
    trainer.register_hooks(
        [
            hooks.IterationTimer(),
            hooks.LRScheduler(scheduler=instantiate(cfg.lr_multiplier)),
            hooks.PeriodicCheckpointer(checkpointer, **cfg.train.checkpointer)
            if comm.is_main_process()
            else None,
            hooks.EvalHook(cfg.train.eval_period, lambda: do_test(cfg, model)),
            hooks.PeriodicWriter(
                default_writers(cfg.train.output_dir, cfg.train.max_iter),
                period=cfg.train.log_period,
            )
            if comm.is_main_process()
            else None,
        ]
    )

    checkpointer.resume_or_load(cfg.train.init_checkpoint, resume=args.resume)
    if args.resume and checkpointer.has_checkpoint():
        # The checkpoint stores the training iteration that just finished, thus we start
        # at the next iteration
        start_iter = trainer.iter + 1
    else:
        start_iter = 0
    trainer.train(start_iter, cfg.train.max_iter)